# ApertureDB

[ApertureDB](https://docs.aperturedata.io) 是一个数据库，可存储、索引和管理多模态数据，如文本、图像、视频、边界框和嵌入，及其关联的元数据。

本笔记本将介绍如何使用 ApertureDB 的嵌入功能。

## 安装 ApertureDB Python SDK

这将安装用于编写 ApertureDB 客户端代码的 [Python SDK](https://docs.aperturedata.io/category/aperturedb-python-sdk)。

In [1]:
%pip install --upgrade --quiet aperturedb

Note: you may need to restart the kernel to use updated packages.


## 运行一个 ApertureDB 实例

继续之前，您应该有一个[已启动并运行的 ApertureDB 实例](https://docs.aperturedata.io/HowToGuides/start/Setup)，并配置您的环境来使用它。
有多种方法可以实现这一点，例如：

```bash
docker run --publish 55555:55555 aperturedata/aperturedb-standalone
adb config create local --active --no-interactive
```

## 下载一些网页文档
这里我们将对一个网页进行一次迷你抓取。

In [2]:
# For loading documents from web
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.aperturedata.io")
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 选择 embeddings 模型

我们想使用 OllamaEmbeddings，所以需要导入必要的模块。

Ollama 可以按照 [文档](https://hub.docker.com/r/ollama/ollama) 中的描述设置为一个 docker 容器，例如：
```bash
# 运行服务器
docker run -d -v ollama:/root/.ollama -p 11434:11434 --name ollama ollama/ollama
# 让服务器加载特定模型
docker exec ollama ollama run llama2
```

In [3]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

## 将文档拆分为小节

我们想将单个文档拆分成多个小节。

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

## 从文档和嵌入创建向量存储

此代码将在 ApertureDB 实例中创建一个向量存储。
在实例中，此向量存储表示为一个“[描述符集](https://docs.aperturedata.io/category/descriptorset-commands)”。
默认情况下，描述符集命名为 `langchain`。以下代码将为每个文档生成嵌入，并将它们作为描述符存储在 ApertureDB 中。由于正在生成嵌入，这需要几秒钟的时间。

In [5]:
from langchain_community.vectorstores import ApertureDB

vector_db = ApertureDB.from_documents(documents, embeddings)

## 选择一个大型语言模型

同样，我们使用为本地处理而设置的 Ollama 服务器。

In [6]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

## 构建 RAG 链

现在我们已经拥有了构建 RAG（检索增强生成）链所需的所有组件。此链执行以下操作：
1. 为用户查询生成嵌入描述符
2. 使用向量存储查找与用户查询相似的文本片段
3. 使用提示模板将用户查询和上下文文档传递给 LLM
4. 返回 LLM 的答案

In [7]:
# Create prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")


# Create a chain that passes documents to an LLM
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


# Treat the vectorstore as a document retriever
retriever = vector_db.as_retriever()


# Create a RAG chain that connects the retriever to the LLM
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

Based on the provided context, ApertureDB can store images. In fact, it is specifically designed to manage multimodal data such as images, videos, documents, embeddings, and associated metadata including annotations. So, ApertureDB has the capability to store and manage images.


## 运行 RAG 链

最后，我们将一个问题传递给链，并得到答案。这将花费几秒钟的时间来运行，因为 LLM 会根据查询和上下文文档生成答案。

In [9]:
user_query = "How can ApertureDB store images?"
response = retrieval_chain.invoke({"input": user_query})
print(response["answer"])

Based on the provided context, ApertureDB can store images in several ways:

1. Multimodal data management: ApertureDB offers a unified interface to manage multimodal data such as images, videos, documents, embeddings, and associated metadata including annotations. This means that images can be stored along with other types of data in a single database instance.
2. Image storage: ApertureDB provides image storage capabilities through its integration with the public cloud providers or on-premise installations. This allows customers to host their own ApertureDB instances and store images on their preferred cloud provider or on-premise infrastructure.
3. Vector database: ApertureDB also offers a vector database that enables efficient similarity search and classification of images based on their semantic meaning. This can be useful for applications where image search and classification are important, such as in computer vision or machine learning workflows.

Overall, ApertureDB provides fl